# Sequential 모델

## 설정

In [ ]:
!pip install tensorflow

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
print(tf.__version__)

2.9.1


## Sequential 모델을 사용하는 경우

`Sequential` 모델은 각 레이어에 **정확히 하나의 입력 텐서와 하나의 출력 텐서**가 있는 **일반 레이어 스택**에 적합합니다.

개략적으로 다음과 같은 `Sequential` 모델은

In [11]:
my_dense = layers.Dense(2, activation="relu") # w.shape() , b.shape()
print(my_dense)
print(my_dense.weights)
x = tf.ones((3, 3))
print(x.numpy().shape)
ret=my_dense(x)
print(my_dense.weights)
print(ret)

[]
(3, 3)
[<tf.Variable 'dense_6/kernel:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.42822504,  0.79465437],
       [ 0.3842728 ,  0.5817646 ],
       [-0.46623665,  0.26333678]], dtype=float32)>, <tf.Variable 'dense_6/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]
tf.Tensor(
[[0.        1.6397557]
 [0.        1.6397557]
 [0.        1.6397557]], shape=(3, 2), dtype=float32)


In [8]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),  # (3,3)(3,2) + (2,) => (3,2)
        layers.Dense(3, activation="relu", name="layer2"),  # (3,2)(2,3) + (3,) => (3,3)
        layers.Dense(4, name="layer3"),                     # (3,3)(3,4) + (4,) => (3,4)
    ]
)
# model.summary()
# Call model on a test input
x = tf.ones((3, 3))
# print(x)
y = model(x)
print(y.shape)
print(y)
model.summary()

(3, 4)
tf.Tensor(
[[ 1.216435    1.1220847   0.14353143 -0.7091525 ]
 [ 1.216435    1.1220847   0.14353143 -0.7091525 ]
 [ 1.216435    1.1220847   0.14353143 -0.7091525 ]], shape=(3, 4), dtype=float32)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (3, 2)                    8         
                                                                 
 layer2 (Dense)              (3, 3)                    9         
                                                                 
 layer3 (Dense)              (3, 4)                    16        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


다음 함수와 동일합니다.

In [12]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
# y = layer3(layer2(layer1(x)))
x = layer1(x)
print(x)
x = layer2(x)
print(x)
x = layer3(x)
print(x)
# y.shape
# model.summary()

tf.Tensor(
[[0.         0.30049294]
 [0.         0.30049294]
 [0.         0.30049294]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[0.2788214  0.         0.26496983]
 [0.2788214  0.         0.26496983]
 [0.2788214  0.         0.26496983]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 0.16995624 -0.15224978  0.14073911 -0.22798349]
 [ 0.16995624 -0.15224978  0.14073911 -0.22798349]
 [ 0.16995624 -0.15224978  0.14073911 -0.22798349]], shape=(3, 4), dtype=float32)


Sequential 모델은 다음의 경우에 **적합하지 않습니다**.

- 모델에 다중 입력 또는 다중 출력이 있습니다
- 레이어에 다중 입력 또는 다중 출력이 있습니다
- 레이어 공유를 해야 합니다
- 비선형 토폴로지를 원합니다(예: 잔류 연결, 다중 분기 모델)

## Sequential 모델 생성하기

레이어의 목록을 Sequential 생성자에 전달하여 Sequential 모델을 만들 수 있습니다.

In [13]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)
print(model)

속한 레이어는 `layers` 속성을 통해 접근할 수 있습니다.

In [14]:
model.layers

`add()` 메서드를 통해 Sequential 모델을 점진적으로 작성할 수도 있습니다.

In [15]:
a = [1,2,3]
print(a)

[1, 2, 3]


In [18]:
a = []
a.append(1)
print(a)
a.append(2)
print(a)
a.append(3)
print(a)

[1]
[1, 2]
[1, 2, 3]


In [22]:
model = keras.Sequential()
print(model.layers)
model.add(layers.Dense(2, activation="relu", name="layer1"))
print(model.layers)
model.add(layers.Dense(3, activation="relu", name="layer2"))
print(model.layers)
model.add(layers.Dense(4, name="layer3"))
print(model.layers)

[]
[<keras.layers.core.dense.Dense object at 0x000001F7E0933F40>, <keras.layers.core.dense.Dense object at 0x000001F7ED9F6F10>]
[<keras.layers.core.dense.Dense object at 0x000001F7E0933F40>, <keras.layers.core.dense.Dense object at 0x000001F7ED9F6F10>, <keras.layers.core.dense.Dense object at 0x000001F7E0933430>]


In [23]:
for data in a:
    print(data)

1
2
3


In [24]:
for layer in model.layers:
    print(layer.name)

layer1
layer2
layer3


레이어를 제거하는 `pop()` 메서드도 있습니다. Sequential 모델은 레이어의 리스트와 매우 유사하게 동작합니다.

In [26]:
model.pop()
print(len(model.layers))  # 2
for layer in model.layers:
    print(layer.name)

2
layer1
layer2


또한 Sequential 생성자는 Keras의 모든 레이어 또는 모델과 마찬가지로 `name` 인수를 허용합니다. 이것은 의미론적으로 유의미한 이름으로 TensorBoard 그래프에 주석을 달 때 유용합니다.

In [28]:
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))
print(model.name)
for layer in model.layers:
    print(layer.name)

my_sequential
layer1
layer2
layer3


## 미리 입력 형상 지정하기

일반적으로 Keras의 모든 레이어는 가중치를 만들려면 입력의 형상을 알아야 합니다. 따라서 다음과 같은 레이어를 만들면 처음에는 가중치가 없습니다.

In [29]:
layer = layers.Dense(3)   # w=(?,3) b=(3,)
layer.weights  # Empty

[]

가중치는 모양이 입력의 형상에 따라 달라지기 때문에 입력에서 처음 호출될 때 가중치를 만듭니다.

In [30]:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)   # (1,4)(4,3)+(3,)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_3/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[ 0.87066984, -0.21357107,  0.3322096 ],
        [ 0.27695417,  0.538417  ,  0.0620122 ],
        [-0.04509944,  0.42110765, -0.0055148 ],
        [ 0.06132042,  0.84137523, -0.8253554 ]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

당연히 이것은 Sequential 모델에도 적용됩니다. 입력 형상이 없는 Sequential 모델을 인스턴스화할 때는 "빌드"되지 않습니다. 가중치가 없습니다(그리고 `model.weights`를 호출하면 오류가 발생함). 모델에 처음 입력 데이터가 표시되면 가중치가 생성됩니다.

In [33]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),  # (1,4)(4,2)+(2,) => (1,2)
        layers.Dense(3, activation="relu"),  # (1,2)(2,3)+(3,) => (1,3)
        layers.Dense(4),                     # (1,3)(3,4)+(4,) => (1,4)
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6


모델이 "빌드"되면, 그 내용을 표시하기 위해 `summary()` 메서드를 호출할 수 있습니다.

In [34]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (1, 2)                    10        
                                                                 
 dense_11 (Dense)            (1, 3)                    9         
                                                                 
 dense_12 (Dense)            (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


그러나 현재 출력 형상을 포함하여 지금까지 모델의 요약을 표시할 수 있도록 Sequential 모델을 점진적으로 빌드할 때 매우 유용할 수 있습니다. 이 경우 `Input` 객체를 모델에 전달하여 모델의 시작 형상을 알 수 있도록 모델을 시작해야 합니다.

In [36]:
# x = tf.ones((1, 4))
# print(x)
x_spec = keras.Input(shape=(4,))
print(x_spec)

KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [38]:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))   # (None,4)(4,2)+(2,) => (None,2)

model.summary()
model.weights

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense_14/kernel:0' shape=(4, 2) dtype=float32, numpy=
 array([[ 0.9835026 , -0.49841833],
        [ 0.5286529 ,  0.39257598],
        [ 0.29013896,  0.9039359 ],
        [-0.7185271 ,  0.19386053]], dtype=float32)>,
 <tf.Variable 'dense_14/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

`Input` 객체는 레이어가 아니므로 `model.layers`의 일부로 표시되지 않습니다.

In [39]:
model.layers

간단한 대안은 첫 번째 레이어에 `input_shape` 인수를 전달하는 것입니다.

In [40]:
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu", input_shape=(4,)))  # (None,4)(4,2)+(2,) => (None,2)

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


이처럼 사전 정의된 입력 모양으로 빌드된 모델은 항상 가중치를 가지며(데이터를 보기 전에도) 항상 정의된 출력 형상을 갖습니다.

일반적으로 Sequential 모델의 입력 형상을 알고 있는 경우 항상 Sequential 모델의 입력 형상을 지정하는 것이 좋습니다.

## 일반적인 디버깅 워크플로우: `add()` + `summary()`

새로운 Sequential 아키텍처를 구축할 때는 `add()` 하여 레이어를 점진적으로 쌓고 모델 요약을 자주 인쇄하는 것이 유용합니다. 예를 들어 `Conv2D` 및 `MaxPooling2D` 레이어의 스택이 이미지 특성 맵을 다운 샘플링 하는 방법을 모니터링할 수 있습니다.

In [44]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(layers.Conv2D(32, 5, strides=2, activation="relu"))  # (5,5,3,32) => (N-F)/S+1 => (None,123,123,32)
model.add(layers.Conv2D(32, 3, activation="relu"))             # (3,3,32,32)=>(None,121,121,32)
model.add(layers.MaxPooling2D(3))                              # (None,121,121,32) => (None,40,40,32)

# # Can you guess what the current output shape is at this point? Probably not.
# # Let's just print it:
model.summary()

# # The answer was: (40, 40, 32), so we can keep downsampling...

model.add(layers.Conv2D(32, 3, activation="relu")) # (3,3,32,32)=>(None,38,38,32)
model.add(layers.Conv2D(32, 3, activation="relu")) # (3,3,32,32)=>(None,36,36,32)
model.add(layers.MaxPooling2D(3))                  #            =>(None,12,12,32)
model.add(layers.Conv2D(32, 3, activation="relu")) # (3,3,32,32)=>(None,10,10,32)
model.add(layers.Conv2D(32, 3, activation="relu")) # (3,3,32,32)=>(None,8,8,32)
model.add(layers.MaxPooling2D(2))                  #            =>(None,4,4,32)

# # And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())  # (None,4,4,32) => (None,32)

model.summary()
# Finally, we add a classification layer.
model.add(layers.Dense(10))  # (None,32)(32,10) + (10,)  => (None,10)
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 123, 123, 32)      2432      
                                                                 
 conv2d_15 (Conv2D)          (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 40, 40, 32)       0         
 2D)                                                             
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 123, 123, 32)      2432      
                          

매우 실용적이죠?


## 모델이 완성되면 해야 할 일

모델 아키텍처가 준비되면 다음을 수행할 수 있습니다.

- 모델을 훈련시키고 평가하며 추론을 실행합니다. [내장 루프를 사용한 훈련 및 평가 가이드](https://www.tensorflow.org/guide/keras/train_and_evaluate/)를 참조하세요.
- 모델을 디스크에 저장하고 복구합니다. [직렬화 및 저장 가이드](https://www.tensorflow.org/guide/keras/save_and_serialize/)를 참조하세요.
- 다중 GPU를 활용하여 모델의 훈련 속도를 향상합니다. [다중 GPU 및 분산 훈련 가이드](distributed_training)를 참조하세요.

## Sequential 모델을 사용한 특성 추출

Sequential 모델이 빌드되면 [Functional API 모델](https://www.tensorflow.org/guide/keras/functional/)처럼 동작합니다. 이는 모든 레이어가 `input` 및 `output` 속성을 갖는다는 것을 의미합니다. 이러한 속성을 사용하면 Sequential 모델 내의 모든 중간 레이어들의 출력을 추출하는 모델을 빠르게 생성하는 등 깔끔한 작업을 수행할 수 있습니다.

In [46]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"), # (1,123,123,32)
        layers.Conv2D(32, 3, activation="relu"),            # (1,121,121,32)
        layers.Conv2D(32, 3, activation="relu"),            # (1,119,119,32)
    ]
)
initial_model.summary()
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)
for feature in features:
    print(feature.shape)

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_23 (Conv2D)          (None, 123, 123, 32)      2432      
                                                                 
 conv2d_24 (Conv2D)          (None, 121, 121, 32)      9248      
                                                                 
 conv2d_25 (Conv2D)          (None, 119, 119, 32)      9248      
                                                                 
Total params: 20,928
Trainable params: 20,928
Non-trainable params: 0
_________________________________________________________________
(1, 123, 123, 32)
(1, 121, 121, 32)
(1, 119, 119, 32)


다음은 한 레이어에서 특성만 추출하는 것과 유사한 예입니다.

In [ ]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)
features.shape

## Sequential 모델을 통한 전이 학습

전이 학습은 모델에서 맨 아래 레이어를 동결하고 맨 위 레이어만 훈련하는 것으로 구성됩니다. 익숙하지 않은 경우, [전이 학습 가이드](https://www.tensorflow.org/guide/keras/transfer_learning/)를 읽어보세요.

다음은 Sequential 모델과 관련된 두 가지 일반적인 전이 학습 청사진입니다.

먼저 Sequential 모델이 있고 마지막 모델을 제외한 모든 레이어를 동결하려고 한다고 가정합니다. 이 경우 다음과 같이 단순히 `model.layers`를 반복하고 마지막 레이어를 제외하고 각 레이어에서 `layer.trainable = False`를 설정합니다.

```python
model = keras.Sequential([<br>    keras.Input(shape=(784))<br>    layers.Dense(32, activation='relu'),<br>    layers.Dense(32, activation='relu'),<br>    layers.Dense(32, activation='relu'),<br>    layers.Dense(10),<br>])<br> <br># Presumably you would want to first load pre-trained weights.<br>model.load_weights(...)<br> <br># Freeze all layers except the last one.<br>for layer in model.layers[:-1]:<br>  layer.trainable = False<br> <br># Recompile and train (this will only update the weights of the last layer).<br>model.compile(...)<br>model.fit(...)
```

또 다른 일반적인 청사진은 다음과 같이 Sequential 모델을 사용하여 사전 훈련된 모델과 새로 초기화된 분류 레이어를 쌓는 것입니다.

```python
# Load a convolutional base with pre-trained weights<br>base_model = keras.applications.Xception(<br>    weights='imagenet',<br>    include_top=False,<br>    pooling='avg')<br> <br># Freeze the base model<br>base_model.trainable = False<br> <br># Use a Sequential model to add a trainable classifier on top<br>model = keras.Sequential([<br>    base_model,<br>    layers.Dense(1000),<br>])<br> <br># Compile & train<br>model.compile(...)<br>model.fit(...)
```

전이 학습을 한다면 아마도 이 두 가지 패턴을 자주 사용하게 될 것입니다.

이것이 Sequential 모델에 대해 알아야 할 전부입니다!

Keras에서 모델을 빌드하는 방법에 대한 자세한 내용은 다음을 참조하세요.

- [Functional API 가이드](https://www.tensorflow.org/guide/keras/functional/)
- [하위 클래스화를 통한 새 레이어 및 모델 생성 가이드](https://www.tensorflow.org/guide/keras/custom_layers_and_models/)